<a href="https://colab.research.google.com/github/FajarKKP/Bottle_cap_color_classifier/blob/main/Bottle_cap_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This will be the jupyter notebook where the bottle cap classifier is used

Dataset prep.

So I have to re-label the datset with the following label:

0 = other
1 = light blue
2 = dark blue

I relabel it in roboflow due to its ease of use and the complimentary tools it have for data preparation.

When using roboflow, there is the option to prep the dataset (from preprocessing step until augmentation). Usually this is done when we are training on the model, not before. There are pros and cons on preprocess + augment dataset before or on the fly. The main pros is usually resource related, because we basically skip that step during training and has been prep beforehand. The cons is that the dataset will inflexible, if it does not fit what we are trying to achieve, the process is useless and we have to prep it again on the fly.

For this project, I do the preprocessing and augmentation beforehand. My decision is based on the fact that I need all the time to train the best performing model. By skipping the preprocess and data augmentation on the fly, I can start training faster. Not only that, the preprocess and data augmentation that I am going to do on the fly has been done during the dataset prep step on roboflow. So preprocessing + augmenting data that has been done is a redundant task.

For the preprocessing, I apply the Auto-orient step. It is used to re-orient images that may be saved in EXIF metadata. This is usefull to prevent bounding box misalignment due to EXIF metadata save.

For the data augmentation, i mainly do the augment related to position such as rotation and shear. I do not do augmentation that is color related such as brightness and noise addition due to small dataset + color detection is also the main key in this project.




Based on initial search on the web, I will be testing on Yolov5 and Yolov8. Each will be tested on their nano model. The main reason on choosing those models are their reputation on solving this type of problem while having a small size (under 3.5M parameter), which is very appropriate to be implemented on edge devices (ex. raspberry pi 5).

YOLOv5 will serve as the minimum benchmark. While slightly older, it has proven effective in practical applications. Its smaller variants are advantageous for deployment on edge devices, and many existing object detection solutions are based on this model.

YOLOv8 is a practical choice for object detection, offering a strong balance of accuracy and speed. Newer YOLO versions provide incremental improvements but are generally larger, making YOLOv8 well-suited for real-world deployment and edge applications.


In [3]:
# Set up and install yolov5
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17739, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 17739 (delta 57), reused 31 (delta 31), pack-reused 17643 (from 4)
Receiving objects: 100% (17739/17739), 17.11 MiB | 18.30 MiB/s, done.
Resolving deltas: 100% (12044/12044), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.1 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
# check if it is folder is read
import glob
import os

imgs = glob.glob('/content/drive/MyDrive/bottle_cap_project/Datset/train/images/*')
labels = [x.replace('/images/', '/labels/').replace('.jpg', '.txt') for x in imgs]

missing = [l for l in labels if not os.path.exists(l)]

len(imgs), len(labels), len(missing)


(21, 21, 0)

In [16]:
# Train
%cd /content/yolov5

!python train.py \
    --img 416 \
    --batch 4 \
    --epochs 200 \
    --save-period 10 \
    --data /content/drive/MyDrive/bottle_cap_project/Datset/data.yaml \
    --weights yolov5s.pt \
    --project /content/drive/MyDrive/bottle_cap_project/Result/train \
    --name bottlecap_yolo5_small \
    --cache


/content/yolov5
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-11-17 15:24:09.408036: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763393049.449460   41052 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763393049.458466   41052 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763393049.486084   41052 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763393049.486137   41052 computation_placer.cc:177] computation placer a

yolov8 nano
Now this will be the section where I will train on yolov8 nano

In [26]:
# Install YOLOv8 (Ultralytics)
!pip install ultralytics

# Import the library
from ultralytics import YOLO

# Load a YOLOv8 nano model (pretrained)
model = YOLO("yolov8n.pt")  # 'n' is nano

In [27]:
# Train on your dataset
model.train(
    data="/content/drive/MyDrive/bottle_cap_project/Datset/data.yaml",  # path to your data.yaml
    imgsz=416,
    batch=8,
    epochs=150,
    project="/content/drive/MyDrive/bottle_cap_project/Result/train",
    name="bottlecap_yolov8n",
    cache=True
)

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/bottle_cap_project/Datset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=bottlecap_yolov8n, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patien

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78ab600fdca0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [12]:
# Export yolo5 model for raspberry PI in ONNX
!python export.py \
    --weights /content/drive/MyDrive/bottle_cap_project/Result/train/bottlecap_yolo5_small_4150/weights/best.pt \
    --imgsz 416 \
    --include onnx \
    --simplify \
    --device cpu \
    --dynamic \
    --optimize




export: data=data/coco128.yaml, weights=['/content/drive/MyDrive/bottle_cap_project/Result/train/bottlecap_yolo5_small_4150/weights/best.pt'], imgsz=[416], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=True, int8=False, per_tensor=False, dynamic=True, cache=, simplify=True, mlmodel=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-448-gdeec5e45 Python-3.12.12 torch-2.8.0+cu126 CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from /content/drive/MyDrive/bottle_cap_project/Result/train/bottlecap_yolo5_small_4150/weights/best.pt with output shape (1, 10647, 8) (13.6 MB)

ONNX: starting export with onnx 1.20.0rc1...
/content/yolov5/export.py:340: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONN

In [16]:
from ultralytics import YOLO

# Load your trained YOLOv8 Nano model
model = YOLO("/content/drive/MyDrive/bottle_cap_project/Result/train/bottlecap_yolov8n_8200/weights/best.pt")

# Export to ONNX
model.export(
    format="onnx",   # export format
    imgsz=416,       # image size for export (keep same as training or as needed for deployment)
    simplify=True,   # simplify ONNX graph (optional but recommended)
    dynamic=True     # dynamic input shapes for flexibility
)


Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/bottle_cap_project/Result/train/bottlecap_yolov8n_8200/weights/best.pt' with input shape (1, 3, 416, 416) BCHW and output shape(s) (1, 7, 3549) (5.9 MB)

ONNX: starting export with onnx 1.19.1 opset 22...
ONNX: slimming with onnxslim 0.1.74...
ONNX: export success ✅ 3.0s, saved as '/content/drive/MyDrive/bottle_cap_project/Result/train/bottlecap_yolov8n_8200/weights/best.onnx' (12.0 MB)

Export complete (3.4s)
Results saved to /content/drive/MyDrive/bottle_cap_project/Result/train/bottlecap_yolov8n_8200/weights
Predict:         yolo predict task=detect model=/content/drive/MyDrive/bottle_cap_project/Result/train/bottlecap_yolov8n_8200/weights/best.onnx imgsz=416  
Validate:        yolo val task=detect model=/content/drive/MyDrive/bottle_cap_project/Result/train/bottlecap_yolo

'/content/drive/MyDrive/bottle_cap_project/Result/train/bottlecap_yolov8n_8200/weights/best.onnx'

For testing or inference using the test dataset, i will use the model that is saved on the onnx version.

The reasons are:
Deployment flexibility: ONNX can run on many platforms (TensorRT, OpenVINO, C++, C#, even some mobile/edge devices) without needing PyTorch.

Hardware optimization: ONNX runtimes can be faster, especially on devices with limited resources like Raspberry Pi 5.

Standardized format: Makes it easier to integrate with production pipelines or convert to other formats (e.g., TensorRT, CoreML).

Testing will be done on my locally (single core nvidia geforce gtx 1050)